In [13]:
%display latex

In [14]:
from sage.combinat.q_analogues import *
from sage.combinat.sf.sfa import *
from itertools import product

In [15]:
Sym = SymmetricFunctions(QQ)
Sym.inject_shorthands()

Defining e as shorthand for Symmetric Functions over Rational Field in the elementary basis
Defining f as shorthand for Symmetric Functions over Rational Field in the forgotten basis
Defining h as shorthand for Symmetric Functions over Rational Field in the homogeneous basis
Defining m as shorthand for Symmetric Functions over Rational Field in the monomial basis
Defining p as shorthand for Symmetric Functions over Rational Field in the powersum basis
Defining s as shorthand for Symmetric Functions over Rational Field in the Schur basis


In [16]:
Qqt=QQ['q','t']
Rqt=FractionField(Qqt)
Sym = SymmetricFunctions(Rqt)
S = SymmetricFunctions(QQ)
Sym.inject_shorthands(verbose=False)
H = Sym.macdonald().Ht()
t=H.t
q=H.q

s.set_print_style('length')
e.set_print_style('length')
H.set_print_style('length')

One=s([])
X=tensor([s[1],s[0]])
Y=tensor([s[0],s[1]])
M=(1-q)*(1-t)
def mystr(i): 
    if i<10: 
        return str(i) 
    else: 
        return ''.join([str(i),"."])

In [17]:
def InSchur(F):
    try:
        return s(Sym(S.from_polynomial(Rqt(F).numerator()))).restrict_partition_lengths(2,exact=false)
    except:
        return F

In [25]:
#Moves 1 from position j to position j+1 in v
def g(j,v):
    v[j]=v[j]-1
    v[j+1]=v[j+1]+1
    return v

#Returns the maximal chain of the poset for a=4, b, and k
def maxchain(b,k):
    L=[]
    if k==0:
        L=[[3]]
    else:
        v=[2]+[0]*(k-1)+[1]+[0]*k
        L.append(copy(v))
        for i in range(2):
            for j in range(2*k):
                v=g(j,v)
                L.append(copy(v))
    return L

#Adds n 0's at the beginning and the end of the vectors of L
def embed(L,n):
    for i in range(len(L)):
        L[i]=[0]*n+L[i]+[0]*n
    return L

#Returns the n^th set of chains from the maximal one for a=4 and b,
#n=0 is the set of maximal chains, 0=< n =< b//2
def chains(b,n):
    L=[]
    for k in range(b//2+1-n):
        K=maxchain(b,k)
        L=L+copy(K)
    embed(L,n)
    return L

#Returns L which contains all the sets of chains in our decomposition of all the posets,
#L[n] being the set of the n^th chains in all the posets (0 being the maximal chains)
def setChains(b):
    L=[]
    for n in range(b//2+1):
        L=L+[chains(b,n)]
    return L

# Polynôme associé à un vecteur (alpha,delta,beta) fixé
def monomials(M,b,s):
    i=M[s]
    r=1
    k=len(i)//2
    for j in range(k):
        r=r*(q^(b-j)*t^j)^i[j] *(t^(b-j)*q^j)^i[2*k-j]
    r=r*(q*t)^(k*(i[k]+1))*h([(b-2*k)*(i[k]+1)]).expand(2,alphabet='q,t')
    return r

# Somme des polynômes associés aux vecteurs (alpha,delta,beta) pour k fixé
def polynomials(M,b):
    P=0
    for s in range(len(M)):
        P=P+monomials(M,b,s)
    return P

#Tests the positivity of every set of chains,
#If it returns K=[], the decomposition is Schur positive
def testPos(b):
    L=setChains(b)
    K=[]
    for n in range(len(L)):
        if InSchur(polynomials(L[n],b)).is_schur_positive()==false:
            K=K+[[n,L[n]]]
    return K

#Return the Schur decomposition for a=4 and b
def Coef(b):
    L=setChains(b)
    P=0
    for n in range (len(L)):
        P=P+InSchur(polynomials(L[n],b))
    return P

#Return the Schur decomposition of every set of chains for a=4 and b
#P[n] is the decomposition for the n^th set of chains
def CoefChains(b):
    L=setChains(b)
    P=[]
    for n in range (len(L)):
        P=P+[InSchur(polynomials(L[n],b))]
    return P

#Returns the n^th set of chains from the maximal one for a=4 and b, for 0=< n =< b//2
#Each chain is in a different coordonate, L[k] is the n^th chain in the k^th poset 
def chainsAux(b,n):
    L=[]
    for k in range(b//2+1-n):
        K=maxchain(b,k)
        L=L+[copy(K)]
    for i in range(len(L)):
        embed(L[i],n)
    return L

#Returns L which contains all the sets of chains in our decomposition of all the posets,
#L[n] being the set of the n^th chains in all the posets separated into chains and
#L[n][k] being the n^th chain of the k^th poset
def setChainsAux(b):
    L=[]
    for i in range(b//2+1):
        L=L+[chainsAux(b,i)]
    return L

#Return the Schur decomposition of every set of chains for a=4 and b
#P[n][k] is the decomposition for the n^th chain of the k^th poset
def CoefChainsAux(b):
    L=setChainsAux(b)
    P=[]
    for n in range(len(L)):
        for k in range(len(L[n])):
            K=[]
            K=K+[InSchur(polynomials(L[n][k],b))]
            P=P+[(n,k),copy(K)]
    return P

#Return the Schur decomposition of every "pair" of vectors of the n^th chains for a=4 and b
#It's pairs if the vector is not symmetric in order to create a symmetric polynomial.
def CoefVector(b,n):
    L=chainsAux(b,n)
    P=[]
    for i in range(len(L)):
        if len(L[i])%2==0:
            k=len(L[i])//2
            p=true
        else:
            k=len(L[i])//2
            p=false
        if p==true:
            for j in range(k):
                P.append([(L[i][j]),InSchur(monomials(L[i],b,j)+monomials(L[i],b,len(L[i])-j-1))])
        else:
            for j in range(k):
                P.append([(L[i][j]),InSchur(monomials(L[i],b,j)+monomials(L[i],b,len(L[i])-j-1))])
            P.append([(L[i][k]),InSchur(monomials(L[i],b,k))])
    return P

In [19]:
CoefVector(2,0)

[[3],
 s[8],
 [2, 1, 0],
 s[6, 2] - s[5, 3],
 [1, 2, 0],
 s[5, 3] - s[4, 4],
 [1, 1, 1],
 s[4, 4]]

In [8]:
CoefVector(2,1)

[[0, 3, 0], s[4, 4]]

In [9]:
CoefVector(3,0)

[[3],
 s[12],
 [2, 1, 0],
 s[10, 2] - s[7, 5],
 [1, 2, 0],
 s[9, 3] + s[6, 6],
 [1, 1, 1],
 s[7, 5]]

In [10]:
CoefVector(3,1)

[[0, 3, 0], s[8, 4]]

In [11]:
CoefVector(4,0)

[[3],
 s[16],
 [2, 1, 0],
 s[14, 2] - s[9, 7],
 [1, 2, 0],
 s[13, 3] + s[9, 7],
 [1, 1, 1],
 s[10, 6],
 [2, 0, 1, 0, 0],
 s[12, 4] - s[11, 5],
 [1, 1, 1, 0, 0],
 s[11, 5] - s[10, 6],
 [1, 0, 2, 0, 0],
 s[10, 6] - s[9, 7],
 [1, 0, 1, 1, 0],
 s[9, 7] - s[8, 8],
 [1, 0, 1, 0, 1],
 s[8, 8]]

In [12]:
CoefVector(4,1)

[[0, 3, 0],
 s[12, 4],
 [0, 2, 1, 0, 0],
 s[10, 6] - s[9, 7],
 [0, 1, 2, 0, 0],
 s[9, 7] - s[8, 8],
 [0, 1, 1, 1, 0],
 s[8, 8]]

In [82]:
CoefVector(4,2)

[[0, 0, 3, 0, 0], s[8, 8]]

In [83]:
CoefVector(5,0)

[[3],
 s[20],
 [2, 1, 0],
 s[18, 2] - s[11, 9],
 [1, 2, 0],
 s[17, 3] + s[12, 8],
 [1, 1, 1],
 s[13, 7],
 [2, 0, 1, 0, 0],
 s[16, 4] - s[13, 7],
 [1, 1, 1, 0, 0],
 s[15, 5] - s[12, 8],
 [1, 0, 2, 0, 0],
 s[14, 6] - s[10, 10],
 [1, 0, 1, 1, 0],
 s[12, 8] + s[10, 10],
 [1, 0, 1, 0, 1],
 s[11, 9]]

In [84]:
CoefVector(5,1)

[[0, 3, 0],
 s[16, 4],
 [0, 2, 1, 0, 0],
 s[14, 6] - s[11, 9],
 [0, 1, 2, 0, 0],
 s[13, 7] + s[10, 10],
 [0, 1, 1, 1, 0],
 s[11, 9]]

In [85]:
CoefVector(5,2)

[[0, 0, 3, 0, 0], s[12, 8]]

In [86]:
CoefVector(6,0)

[[3],
 s[24],
 [2, 1, 0],
 s[22, 2] - s[13, 11],
 [1, 2, 0],
 s[21, 3] + s[15, 9],
 [1, 1, 1],
 s[16, 8],
 [2, 0, 1, 0, 0],
 s[20, 4] - s[15, 9],
 [1, 1, 1, 0, 0],
 s[19, 5] - s[14, 10],
 [1, 0, 2, 0, 0],
 s[18, 6] + s[12, 12],
 [1, 0, 1, 1, 0],
 s[15, 9] + s[13, 11],
 [1, 0, 1, 0, 1],
 s[14, 10],
 [2, 0, 0, 1, 0, 0, 0],
 s[18, 6] - s[17, 7],
 [1, 1, 0, 1, 0, 0, 0],
 s[17, 7] - s[16, 8],
 [1, 0, 1, 1, 0, 0, 0],
 s[16, 8] - s[15, 9],
 [1, 0, 0, 2, 0, 0, 0],
 s[15, 9] - s[14, 10],
 [1, 0, 0, 1, 1, 0, 0],
 s[14, 10] - s[13, 11],
 [1, 0, 0, 1, 0, 1, 0],
 s[13, 11] - s[12, 12],
 [1, 0, 0, 1, 0, 0, 1],
 s[12, 12]]

In [87]:
CoefVector(6,1)

[[0, 3, 0],
 s[20, 4],
 [0, 2, 1, 0, 0],
 s[18, 6] - s[13, 11],
 [0, 1, 2, 0, 0],
 s[17, 7] + s[13, 11],
 [0, 1, 1, 1, 0],
 s[14, 10],
 [0, 2, 0, 1, 0, 0, 0],
 s[16, 8] - s[15, 9],
 [0, 1, 1, 1, 0, 0, 0],
 s[15, 9] - s[14, 10],
 [0, 1, 0, 2, 0, 0, 0],
 s[14, 10] - s[13, 11],
 [0, 1, 0, 1, 1, 0, 0],
 s[13, 11] - s[12, 12],
 [0, 1, 0, 1, 0, 1, 0],
 s[12, 12]]

In [88]:
CoefVector(6,2)

[[0, 0, 3, 0, 0],
 s[16, 8],
 [0, 0, 2, 1, 0, 0, 0],
 s[14, 10] - s[13, 11],
 [0, 0, 1, 2, 0, 0, 0],
 s[13, 11] - s[12, 12],
 [0, 0, 1, 1, 1, 0, 0],
 s[12, 12]]

In [90]:
CoefVector(6,3)

[[0, 0, 0, 3, 0, 0, 0], s[12, 12]]

In [91]:
CoefVector(7,0)

[[3],
 s[28],
 [2, 1, 0],
 s[26, 2] - s[15, 13],
 [1, 2, 0],
 s[25, 3] + s[18, 10],
 [1, 1, 1],
 s[19, 9],
 [2, 0, 1, 0, 0],
 s[24, 4] - s[17, 11],
 [1, 1, 1, 0, 0],
 s[23, 5] - s[16, 12],
 [1, 0, 2, 0, 0],
 s[22, 6] + s[15, 13],
 [1, 0, 1, 1, 0],
 s[18, 10] + s[16, 12],
 [1, 0, 1, 0, 1],
 s[17, 11],
 [2, 0, 0, 1, 0, 0, 0],
 s[22, 6] - s[19, 9],
 [1, 1, 0, 1, 0, 0, 0],
 s[21, 7] - s[18, 10],
 [1, 0, 1, 1, 0, 0, 0],
 s[20, 8] - s[17, 11],
 [1, 0, 0, 2, 0, 0, 0],
 s[19, 9] - s[15, 13],
 [1, 0, 0, 1, 1, 0, 0],
 s[17, 11] - s[14, 14],
 [1, 0, 0, 1, 0, 1, 0],
 s[16, 12] + s[14, 14],
 [1, 0, 0, 1, 0, 0, 1],
 s[15, 13]]

In [92]:
CoefVector(7,1)

[[0, 3, 0],
 s[24, 4],
 [0, 2, 1, 0, 0],
 s[22, 6] - s[15, 13],
 [0, 1, 2, 0, 0],
 s[21, 7] + s[16, 12],
 [0, 1, 1, 1, 0],
 s[17, 11],
 [0, 2, 0, 1, 0, 0, 0],
 s[20, 8] - s[17, 11],
 [0, 1, 1, 1, 0, 0, 0],
 s[19, 9] - s[16, 12],
 [0, 1, 0, 2, 0, 0, 0],
 s[18, 10] - s[14, 14],
 [0, 1, 0, 1, 1, 0, 0],
 s[16, 12] + s[14, 14],
 [0, 1, 0, 1, 0, 1, 0],
 s[15, 13]]

In [93]:
CoefVector(7,2)

[[0, 0, 3, 0, 0],
 s[20, 8],
 [0, 0, 2, 1, 0, 0, 0],
 s[18, 10] - s[15, 13],
 [0, 0, 1, 2, 0, 0, 0],
 s[17, 11] + s[14, 14],
 [0, 0, 1, 1, 1, 0, 0],
 s[15, 13]]

In [94]:
CoefVector(7,3)

[[0, 0, 0, 3, 0, 0, 0], s[16, 12]]

In [95]:
CoefVector(8,0)

[[3],
 s[32],
 [2, 1, 0],
 s[30, 2] - s[17, 15],
 [1, 2, 0],
 s[29, 3] + s[21, 11],
 [1, 1, 1],
 s[22, 10],
 [2, 0, 1, 0, 0],
 s[28, 4] - s[19, 13],
 [1, 1, 1, 0, 0],
 s[27, 5] - s[18, 14],
 [1, 0, 2, 0, 0],
 s[26, 6] + s[18, 14],
 [1, 0, 1, 1, 0],
 s[21, 11] + s[19, 13],
 [1, 0, 1, 0, 1],
 s[20, 12],
 [2, 0, 0, 1, 0, 0, 0],
 s[26, 6] - s[21, 11],
 [1, 1, 0, 1, 0, 0, 0],
 s[25, 7] - s[20, 12],
 [1, 0, 1, 1, 0, 0, 0],
 s[24, 8] - s[19, 13],
 [1, 0, 0, 2, 0, 0, 0],
 s[23, 9] - s[16, 16],
 [1, 0, 0, 1, 1, 0, 0],
 s[20, 12] + s[16, 16],
 [1, 0, 0, 1, 0, 1, 0],
 s[19, 13] + s[17, 15],
 [1, 0, 0, 1, 0, 0, 1],
 s[18, 14],
 [2, 0, 0, 0, 1, 0, 0, 0, 0],
 s[24, 8] - s[23, 9],
 [1, 1, 0, 0, 1, 0, 0, 0, 0],
 s[23, 9] - s[22, 10],
 [1, 0, 1, 0, 1, 0, 0, 0, 0],
 s[22, 10] - s[21, 11],
 [1, 0, 0, 1, 1, 0, 0, 0, 0],
 s[21, 11] - s[20, 12],
 [1, 0, 0, 0, 2, 0, 0, 0, 0],
 s[20, 12] - s[19, 13],
 [1, 0, 0, 0, 1, 1, 0, 0, 0],
 s[19, 13] - s[18, 14],
 [1, 0, 0, 0, 1, 0, 1, 0, 0],
 s[18, 14] - s[17, 15],
 [1, 0, 0, 0, 1, 0, 0, 1, 0],
 s[17, 15] - s[16, 16],
 [1, 0, 0, 0, 1, 0, 0, 0, 1],
 s[16, 16]]

In [96]:
CoefVector(8,1)

[[0, 3, 0],
 s[28, 4],
 [0, 2, 1, 0, 0],
 s[26, 6] - s[17, 15],
 [0, 1, 2, 0, 0],
 s[25, 7] + s[19, 13],
 [0, 1, 1, 1, 0],
 s[20, 12],
 [0, 2, 0, 1, 0, 0, 0],
 s[24, 8] - s[19, 13],
 [0, 1, 1, 1, 0, 0, 0],
 s[23, 9] - s[18, 14],
 [0, 1, 0, 2, 0, 0, 0],
 s[22, 10] + s[16, 16],
 [0, 1, 0, 1, 1, 0, 0],
 s[19, 13] + s[17, 15],
 [0, 1, 0, 1, 0, 1, 0],
 s[18, 14],
 [0, 2, 0, 0, 1, 0, 0, 0, 0],
 s[22, 10] - s[21, 11],
 [0, 1, 1, 0, 1, 0, 0, 0, 0],
 s[21, 11] - s[20, 12],
 [0, 1, 0, 1, 1, 0, 0, 0, 0],
 s[20, 12] - s[19, 13],
 [0, 1, 0, 0, 2, 0, 0, 0, 0],
 s[19, 13] - s[18, 14],
 [0, 1, 0, 0, 1, 1, 0, 0, 0],
 s[18, 14] - s[17, 15],
 [0, 1, 0, 0, 1, 0, 1, 0, 0],
 s[17, 15] - s[16, 16],
 [0, 1, 0, 0, 1, 0, 0, 1, 0],
 s[16, 16]]

In [97]:
CoefVector(8,2)

[[0, 0, 3, 0, 0],
 s[24, 8],
 [0, 0, 2, 1, 0, 0, 0],
 s[22, 10] - s[17, 15],
 [0, 0, 1, 2, 0, 0, 0],
 s[21, 11] + s[17, 15],
 [0, 0, 1, 1, 1, 0, 0],
 s[18, 14],
 [0, 0, 2, 0, 1, 0, 0, 0, 0],
 s[20, 12] - s[19, 13],
 [0, 0, 1, 1, 1, 0, 0, 0, 0],
 s[19, 13] - s[18, 14],
 [0, 0, 1, 0, 2, 0, 0, 0, 0],
 s[18, 14] - s[17, 15],
 [0, 0, 1, 0, 1, 1, 0, 0, 0],
 s[17, 15] - s[16, 16],
 [0, 0, 1, 0, 1, 0, 1, 0, 0],
 s[16, 16]]

In [98]:
CoefVector(8,3)

[[0, 0, 0, 3, 0, 0, 0],
 s[20, 12],
 [0, 0, 0, 2, 1, 0, 0, 0, 0],
 s[18, 14] - s[17, 15],
 [0, 0, 0, 1, 2, 0, 0, 0, 0],
 s[17, 15] - s[16, 16],
 [0, 0, 0, 1, 1, 1, 0, 0, 0],
 s[16, 16]]

In [99]:
CoefVector(8,4)

[[0, 0, 0, 0, 3, 0, 0, 0, 0], s[16, 16]]

In [13]:
CoefVector(1,0)

[[3], s[4]]

In [14]:
CoefVector(2,0)

[[3],
 s[8],
 [2, 1, 0],
 s[6, 2] - s[5, 3],
 [1, 2, 0],
 s[5, 3] - s[4, 4],
 [1, 1, 1],
 s[4, 4]]

In [15]:
CoefVector(3,0)

[[3],
 s[12],
 [2, 1, 0],
 s[10, 2] - s[7, 5],
 [1, 2, 0],
 s[9, 3] + s[6, 6],
 [1, 1, 1],
 s[7, 5]]

In [16]:
CoefVector(4,0)

[[3],
 s[16],
 [2, 1, 0],
 s[14, 2] - s[9, 7],
 [1, 2, 0],
 s[13, 3] + s[9, 7],
 [1, 1, 1],
 s[10, 6],
 [2, 0, 1, 0, 0],
 s[12, 4] - s[11, 5],
 [1, 1, 1, 0, 0],
 s[11, 5] - s[10, 6],
 [1, 0, 2, 0, 0],
 s[10, 6] - s[9, 7],
 [1, 0, 1, 1, 0],
 s[9, 7] - s[8, 8],
 [1, 0, 1, 0, 1],
 s[8, 8]]

In [106]:
CoefVector(5,0)

[[3],
 s[20],
 [2, 1, 0],
 s[18, 2] - s[11, 9],
 [1, 2, 0],
 s[17, 3] + s[12, 8],
 [1, 1, 1],
 s[13, 7],
 [2, 0, 1, 0, 0],
 s[16, 4] - s[13, 7],
 [1, 1, 1, 0, 0],
 s[15, 5] - s[12, 8],
 [1, 0, 2, 0, 0],
 s[14, 6] - s[10, 10],
 [1, 0, 1, 1, 0],
 s[12, 8] + s[10, 10],
 [1, 0, 1, 0, 1],
 s[11, 9]]

In [107]:
CoefVector(6,0)

[[3],
 s[24],
 [2, 1, 0],
 s[22, 2] - s[13, 11],
 [1, 2, 0],
 s[21, 3] + s[15, 9],
 [1, 1, 1],
 s[16, 8],
 [2, 0, 1, 0, 0],
 s[20, 4] - s[15, 9],
 [1, 1, 1, 0, 0],
 s[19, 5] - s[14, 10],
 [1, 0, 2, 0, 0],
 s[18, 6] + s[12, 12],
 [1, 0, 1, 1, 0],
 s[15, 9] + s[13, 11],
 [1, 0, 1, 0, 1],
 s[14, 10],
 [2, 0, 0, 1, 0, 0, 0],
 s[18, 6] - s[17, 7],
 [1, 1, 0, 1, 0, 0, 0],
 s[17, 7] - s[16, 8],
 [1, 0, 1, 1, 0, 0, 0],
 s[16, 8] - s[15, 9],
 [1, 0, 0, 2, 0, 0, 0],
 s[15, 9] - s[14, 10],
 [1, 0, 0, 1, 1, 0, 0],
 s[14, 10] - s[13, 11],
 [1, 0, 0, 1, 0, 1, 0],
 s[13, 11] - s[12, 12],
 [1, 0, 0, 1, 0, 0, 1],
 s[12, 12]]

In [108]:
CoefVector(7,0)

[[3],
 s[28],
 [2, 1, 0],
 s[26, 2] - s[15, 13],
 [1, 2, 0],
 s[25, 3] + s[18, 10],
 [1, 1, 1],
 s[19, 9],
 [2, 0, 1, 0, 0],
 s[24, 4] - s[17, 11],
 [1, 1, 1, 0, 0],
 s[23, 5] - s[16, 12],
 [1, 0, 2, 0, 0],
 s[22, 6] + s[15, 13],
 [1, 0, 1, 1, 0],
 s[18, 10] + s[16, 12],
 [1, 0, 1, 0, 1],
 s[17, 11],
 [2, 0, 0, 1, 0, 0, 0],
 s[22, 6] - s[19, 9],
 [1, 1, 0, 1, 0, 0, 0],
 s[21, 7] - s[18, 10],
 [1, 0, 1, 1, 0, 0, 0],
 s[20, 8] - s[17, 11],
 [1, 0, 0, 2, 0, 0, 0],
 s[19, 9] - s[15, 13],
 [1, 0, 0, 1, 1, 0, 0],
 s[17, 11] - s[14, 14],
 [1, 0, 0, 1, 0, 1, 0],
 s[16, 12] + s[14, 14],
 [1, 0, 0, 1, 0, 0, 1],
 s[15, 13]]

In [109]:
CoefVector(8,0)

[[3],
 s[32],
 [2, 1, 0],
 s[30, 2] - s[17, 15],
 [1, 2, 0],
 s[29, 3] + s[21, 11],
 [1, 1, 1],
 s[22, 10],
 [2, 0, 1, 0, 0],
 s[28, 4] - s[19, 13],
 [1, 1, 1, 0, 0],
 s[27, 5] - s[18, 14],
 [1, 0, 2, 0, 0],
 s[26, 6] + s[18, 14],
 [1, 0, 1, 1, 0],
 s[21, 11] + s[19, 13],
 [1, 0, 1, 0, 1],
 s[20, 12],
 [2, 0, 0, 1, 0, 0, 0],
 s[26, 6] - s[21, 11],
 [1, 1, 0, 1, 0, 0, 0],
 s[25, 7] - s[20, 12],
 [1, 0, 1, 1, 0, 0, 0],
 s[24, 8] - s[19, 13],
 [1, 0, 0, 2, 0, 0, 0],
 s[23, 9] - s[16, 16],
 [1, 0, 0, 1, 1, 0, 0],
 s[20, 12] + s[16, 16],
 [1, 0, 0, 1, 0, 1, 0],
 s[19, 13] + s[17, 15],
 [1, 0, 0, 1, 0, 0, 1],
 s[18, 14],
 [2, 0, 0, 0, 1, 0, 0, 0, 0],
 s[24, 8] - s[23, 9],
 [1, 1, 0, 0, 1, 0, 0, 0, 0],
 s[23, 9] - s[22, 10],
 [1, 0, 1, 0, 1, 0, 0, 0, 0],
 s[22, 10] - s[21, 11],
 [1, 0, 0, 1, 1, 0, 0, 0, 0],
 s[21, 11] - s[20, 12],
 [1, 0, 0, 0, 2, 0, 0, 0, 0],
 s[20, 12] - s[19, 13],
 [1, 0, 0, 0, 1, 1, 0, 0, 0],
 s[19, 13] - s[18, 14],
 [1, 0, 0, 0, 1, 0, 1, 0, 0],
 s[18, 14] - s[17, 15],
 [1, 0, 0, 0, 1, 0, 0, 1, 0],
 s[17, 15] - s[16, 16],
 [1, 0, 0, 0, 1, 0, 0, 0, 1],
 s[16, 16]]

In [110]:
CoefVector(9,0)

[[3],
 s[36],
 [2, 1, 0],
 s[34, 2] - s[19, 17],
 [1, 2, 0],
 s[33, 3] + s[24, 12],
 [1, 1, 1],
 s[25, 11],
 [2, 0, 1, 0, 0],
 s[32, 4] - s[21, 15],
 [1, 1, 1, 0, 0],
 s[31, 5] - s[20, 16],
 [1, 0, 2, 0, 0],
 s[30, 6] + s[21, 15],
 [1, 0, 1, 1, 0],
 s[24, 12] + s[22, 14],
 [1, 0, 1, 0, 1],
 s[23, 13],
 [2, 0, 0, 1, 0, 0, 0],
 s[30, 6] - s[23, 13],
 [1, 1, 0, 1, 0, 0, 0],
 s[29, 7] - s[22, 14],
 [1, 0, 1, 1, 0, 0, 0],
 s[28, 8] - s[21, 15],
 [1, 0, 0, 2, 0, 0, 0],
 s[27, 9] + s[18, 18],
 [1, 0, 0, 1, 1, 0, 0],
 s[23, 13] + s[19, 17],
 [1, 0, 0, 1, 0, 1, 0],
 s[22, 14] + s[20, 16],
 [1, 0, 0, 1, 0, 0, 1],
 s[21, 15],
 [2, 0, 0, 0, 1, 0, 0, 0, 0],
 s[28, 8] - s[25, 11],
 [1, 1, 0, 0, 1, 0, 0, 0, 0],
 s[27, 9] - s[24, 12],
 [1, 0, 1, 0, 1, 0, 0, 0, 0],
 s[26, 10] - s[23, 13],
 [1, 0, 0, 1, 1, 0, 0, 0, 0],
 s[25, 11] - s[22, 14],
 [1, 0, 0, 0, 2, 0, 0, 0, 0],
 s[24, 12] - s[20, 16],
 [1, 0, 0, 0, 1, 1, 0, 0, 0],
 s[22, 14] - s[19, 17],
 [1, 0, 0, 0, 1, 0, 1, 0, 0],
 s[21, 15] - s[18, 18],
 [1, 0, 0, 0, 1, 0, 0, 1, 0],
 s[20, 16] + s[18, 18],
 [1, 0, 0, 0, 1, 0, 0, 0, 1],
 s[19, 17]]

In [111]:
CoefVector(10,0)

[[3],
 s[40],
 [2, 1, 0],
 s[38, 2] - s[21, 19],
 [1, 2, 0],
 s[37, 3] + s[27, 13],
 [1, 1, 1],
 s[28, 12],
 [2, 0, 1, 0, 0],
 s[36, 4] - s[23, 17],
 [1, 1, 1, 0, 0],
 s[35, 5] - s[22, 18],
 [1, 0, 2, 0, 0],
 s[34, 6] + s[24, 16],
 [1, 0, 1, 1, 0],
 s[27, 13] + s[25, 15],
 [1, 0, 1, 0, 1],
 s[26, 14],
 [2, 0, 0, 1, 0, 0, 0],
 s[34, 6] - s[25, 15],
 [1, 1, 0, 1, 0, 0, 0],
 s[33, 7] - s[24, 16],
 [1, 0, 1, 1, 0, 0, 0],
 s[32, 8] - s[23, 17],
 [1, 0, 0, 2, 0, 0, 0],
 s[31, 9] + s[21, 19],
 [1, 0, 0, 1, 1, 0, 0],
 s[26, 14] + s[22, 18],
 [1, 0, 0, 1, 0, 1, 0],
 s[25, 15] + s[23, 17],
 [1, 0, 0, 1, 0, 0, 1],
 s[24, 16],
 [2, 0, 0, 0, 1, 0, 0, 0, 0],
 s[32, 8] - s[27, 13],
 [1, 1, 0, 0, 1, 0, 0, 0, 0],
 s[31, 9] - s[26, 14],
 [1, 0, 1, 0, 1, 0, 0, 0, 0],
 s[30, 10] - s[25, 15],
 [1, 0, 0, 1, 1, 0, 0, 0, 0],
 s[29, 11] - s[24, 16],
 [1, 0, 0, 0, 2, 0, 0, 0, 0],
 s[28, 12] - s[21, 19],
 [1, 0, 0, 0, 1, 1, 0, 0, 0],
 s[25, 15] - s[20, 20],
 [1, 0, 0, 0, 1, 0, 1, 0, 0],
 s[24, 16] + s[20, 20],
 [1, 0, 0, 0, 1, 0, 0, 1, 0],
 s[23, 17] + s[21, 19],
 [1, 0, 0, 0, 1, 0, 0, 0, 1],
 s[22, 18],
 [2, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
 s[30, 10] - s[29, 11],
 [1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0],
 s[29, 11] - s[28, 12],
 [1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0],
 s[28, 12] - s[27, 13],
 [1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0],
 s[27, 13] - s[26, 14],
 [1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0],
 s[26, 14] - s[25, 15],
 [1, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0],
 s[25, 15] - s[24, 16],
 [1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0],
 s[24, 16] - s[23, 17],
 [1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0],
 s[23, 17] - s[22, 18],
 [1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0],
 s[22, 18] - s[21, 19],
 [1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0],
 s[21, 19] - s[20, 20],
 [1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1],
 s[20, 20]]

In [2]:
CoefChainsAux(10)

NameError: name 'CoefChainsAux' is not defined

In [11]:
from tabulate import tabulate

from tabulate import tabulate

In [21]:
table = [["Sun",696000,1989100000],["Earth",6371,5973.6], ["Moon",1737,73.5],["Mars",3390,641.85]]
print(tabulate(table,headers=["Planet","R (km)", "mass (x 10^29 kg)"]))

Planet      R (km)    mass (x 10^29 kg)
--------  --------  -------------------
Sun         696000           1.9891e+09
Earth         6371        5973.6
Moon          1737          73.5
Mars          3390         641.85


In [34]:
P=CoefVector(10,0)

In [36]:
print(tabulate(P))

---------------------------------  ---------------------
[3]                                s[40]
[2, 1, 0]                          s[38, 2] - s[21, 19]
[1, 2, 0]                          s[37, 3] + s[27, 13]
[1, 1, 1]                          s[28, 12]
[2, 0, 1, 0, 0]                    s[36, 4] - s[23, 17]
[1, 1, 1, 0, 0]                    s[35, 5] - s[22, 18]
[1, 0, 2, 0, 0]                    s[34, 6] + s[24, 16]
[1, 0, 1, 1, 0]                    s[27, 13] + s[25, 15]
[1, 0, 1, 0, 1]                    s[26, 14]
[2, 0, 0, 1, 0, 0, 0]              s[34, 6] - s[25, 15]
[1, 1, 0, 1, 0, 0, 0]              s[33, 7] - s[24, 16]
[1, 0, 1, 1, 0, 0, 0]              s[32, 8] - s[23, 17]
[1, 0, 0, 2, 0, 0, 0]              s[31, 9] + s[21, 19]
[1, 0, 0, 1, 1, 0, 0]              s[26, 14] + s[22, 18]
[1, 0, 0, 1, 0, 1, 0]              s[25, 15] + s[23, 17]
[1, 0, 0, 1, 0, 0, 1]              s[24, 16]
[2, 0, 0, 0, 1, 0, 0, 0, 0]        s[32, 8] - s[27, 13]
[1, 1, 0, 0, 1, 0, 0, 0, 0]        s

In [38]:
print(tabulate(CoefVector(12,0)))

---------------------------------------  ---------------------
[3]                                      s[48]
[2, 1, 0]                                s[46, 2] - s[25, 23]
[1, 2, 0]                                s[45, 3] + s[33, 15]
[1, 1, 1]                                s[34, 14]
[2, 0, 1, 0, 0]                          s[44, 4] - s[27, 21]
[1, 1, 1, 0, 0]                          s[43, 5] - s[26, 22]
[1, 0, 2, 0, 0]                          s[42, 6] + s[30, 18]
[1, 0, 1, 1, 0]                          s[33, 15] + s[31, 17]
[1, 0, 1, 0, 1]                          s[32, 16]
[2, 0, 0, 1, 0, 0, 0]                    s[42, 6] - s[29, 19]
[1, 1, 0, 1, 0, 0, 0]                    s[41, 7] - s[28, 20]
[1, 0, 1, 1, 0, 0, 0]                    s[40, 8] - s[27, 21]
[1, 0, 0, 2, 0, 0, 0]                    s[39, 9] + s[27, 21]
[1, 0, 0, 1, 1, 0, 0]                    s[32, 16] + s[28, 20]
[1, 0, 0, 1, 0, 1, 0]                    s[31, 17] + s[29, 19]
[1, 0, 0, 1, 0, 0, 1]                    

In [32]:
for b in range(3,10):
    print("b=",b)
    print(tabulate(CoefVector(b,0), headers=['vectors', 'schur']))
    print('break')

b= 3
vectors    schur
---------  ------------------
[3]        s[12]
[2, 1, 0]  s[10, 2] - s[7, 5]
[1, 2, 0]  s[9, 3] + s[6, 6]
[1, 1, 1]  s[7, 5]
break
b= 4
vectors          schur
---------------  -------------------
[3]              s[16]
[2, 1, 0]        s[14, 2] - s[9, 7]
[1, 2, 0]        s[13, 3] + s[9, 7]
[1, 1, 1]        s[10, 6]
[2, 0, 1, 0, 0]  s[12, 4] - s[11, 5]
[1, 1, 1, 0, 0]  s[11, 5] - s[10, 6]
[1, 0, 2, 0, 0]  s[10, 6] - s[9, 7]
[1, 0, 1, 1, 0]  s[9, 7] - s[8, 8]
[1, 0, 1, 0, 1]  s[8, 8]
break
b= 5
vectors          schur
---------------  --------------------
[3]              s[20]
[2, 1, 0]        s[18, 2] - s[11, 9]
[1, 2, 0]        s[17, 3] + s[12, 8]
[1, 1, 1]        s[13, 7]
[2, 0, 1, 0, 0]  s[16, 4] - s[13, 7]
[1, 1, 1, 0, 0]  s[15, 5] - s[12, 8]
[1, 0, 2, 0, 0]  s[14, 6] - s[10, 10]
[1, 0, 1, 1, 0]  s[12, 8] + s[10, 10]
[1, 0, 1, 0, 1]  s[11, 9]
break
b= 6
vectors                schur
---------------------  ---------------------
[3]                    s[24]
[2, 1,